# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [6]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
#Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
song_files = get_files('data/song_data')


# Select the first song file
first_song_file = song_files[0]

# Read the song file and view the data
with open(first_song_file, 'r') as f:
    song_data = json.load(f)

df = pd.DataFrame([song_data])
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
# Select columns for the songs table: song ID, title, artist ID, year, and duration
songs_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()

songs_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [9]:
# Insert song record
cur.execute(song_table_insert, songs_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [180]:
# Select columns for the songs table: artist ID, name, location, latitude, and longitude
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_latitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

In [181]:
# Insert artists record
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [182]:
#Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
log_files = get_files('data/log_data')


# Select the first song file
first_log_file = log_files[0]

# Read the log file line by line and process each line as a separate JSON object
log_data_list = []
with open(first_log_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        log_data_list.append(data)

log_df = pd.DataFrame(log_data_list)
log_df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [183]:
log_df_filter = log_df.loc[log_df["page"] == "NextSong"]
log_df_filter["ts"] = pd.to_datetime(log_df_filter['ts'], unit='ms')


C:\Users\Qingyuan Sun\AppData\Local\Temp\ipykernel_9452\2700277012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_df_filter["ts"] = pd.to_datetime(log_df_filter['ts'], unit='ms')


In [184]:
# Extract the required time-related data from the 'ts' column
time_data = [
    log_df_filter["ts"],                       
    log_df_filter["ts"].dt.hour,               
    log_df_filter["ts"].dt.day,                
    log_df_filter["ts"].dt.isocalendar().week, 
    log_df_filter["ts"].dt.month,              
    log_df_filter["ts"].dt.year,               
    log_df_filter["ts"].dt.weekday             
]

# Define the column labels
column_labels = ["start_time", "hour", "day", "week", "month", "year", "weekday"]

# Create the time_df DataFrame
time_df = pd.concat(time_data, axis=1)
time_df.columns = column_labels


time_df

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3
8,2018-11-01 21:24:53.796,21,1,44,11,2018,3
9,2018-11-01 21:28:54.796,21,1,44,11,2018,3
10,2018-11-01 21:42:00.796,21,1,44,11,2018,3
12,2018-11-01 21:52:05.796,21,1,44,11,2018,3
13,2018-11-01 21:55:25.796,21,1,44,11,2018,3


In [185]:
# Select columns for the songs table: song ID, title, artist ID, year, and duration
time_df = time_df.values[0].tolist()

cur.execute(time_table_insert, time_df)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [186]:
user_df = log_df[["userId","firstName", "lastName", "gender", "level"]]

In [187]:
# Select columns for the songs table: song ID, title, artist ID, year, and duration
user_df = user_df.values[0].tolist()

cur.execute(user_table_insert, user_df)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:


# Extract necessary columns from log_df
log_df['ts'] = pd.to_datetime(log_df['ts'], unit='ms')  # Convert timestamp to datetime format

a = log_df.dropna()
# Initialize empty list to store songplay data
songplay_data = []

# Iterate through each row in log_df
for index, row in a.iterrows():
    # Get song ID and artist ID using song_select query
    params = (row.song, row.artist, row.length)
    #print(f"Parameters: {params}")  # Debugging line
    cur.execute(song_select, params)
    results = cur.fetchone()
    #print(f"Query Result: {results}")  # Debugging line


    if results:
        
        song_id, artist_id = results
        print(results)
        
    else:
        song_id, artist_id = None, None



    # Append songplay data to songplay_data list
    songplay_data.append([
        row.ts,         # timestamp
        row.userId,     # user ID
        row.level,      # level
        song_id,        # song ID
        artist_id,      # artist ID
        row.sessionId,  # session ID
        row.location,   # location
        row.userAgent   # user agent
    ])

# Insert songplays records
for data in songplay_data:
    cur.execute(songplay_table_insert, data)
    conn.commit()  # Commit the transaction




Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.